In [1]:
%%time
import sys
if not r'G:\PythonProjects\WineRecognition2' in sys.path:
    sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')

import os
import json
import torch
import numpy as np
import pandas as pd
import sklearn_crfsuite
import eli5
from sklearn.model_selection import train_test_split

from data_master import DataGenerator, DataLoader, count_unk_foreach_tag, compute_model_confidence
from nn.utils import CustomDataset, generate_tag_to_ix
from mlflow_utils import log_mlflow_on_train
from features.features import sent2features

D:\ProgramFiles\anaconda3\envs\wine-recognition\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


Wall time: 2.17 s


In [2]:
%%time
DATASET_PATH = r'G:\PythonProjects\WineRecognition2\data\text\halliday_winesearcher_menu_gen_samplesv2\Halliday_WineSearcher_MenuGenSamples.txt'
LSTM_MODEL_PATH = 'G:/PythonProjects/WineRecognition2/artifacts/train/BiLSTM_CRF_17022022_185854/model/data/model.pth'
VOCAB_PATH = 'G:/PythonProjects/WineRecognition2/data/vocabs/Words_Halliday_Wine_AU_WORD_NUMS.json'
CASE_SENSITIVE_VOCAB = False
DICTIONARY_PATH = r"G:\PythonProjects\WineRecognition2\data\dictionaries\Dict-byword_Halliday_Winesearcher_Wine_AU-only_completed_rows"
DATAINFO_PATH = 'G:/PythonProjects/WineRecognition2/data_info.json'
DEVICE = 'cuda'
MODEL_NAME = "CRF_with_LSTM_features"
ALGORITHM = 'lbfgs'
C1 = 0.1
C2 = 0.1
MAX_ITERATIONS = 5
ALL_POSSIBLE_TRANSITIONS = True
TEST_SIZE = 0.2
RUN_NAME = 'Train-LSTMfeatures'
OUTPUT_DIR = r"G:\PythonProjects\WineRecognition2\artifacts\train\test_exp"
START_TIME = ''
USE_NUM2WORDS = True

Wall time: 0 ns


In [3]:
%%time
lstm_model = torch.load(LSTM_MODEL_PATH).to(DEVICE).eval()
freq_dict = DataLoader.load_frequency_dictionary(DICTIONARY_PATH, to_lowercase=True)

with open(VOCAB_PATH, encoding='utf-8') as file:
    word_to_ix = json.load(file)

with open(DATAINFO_PATH) as file:
    keys = json.load(file)['keys']['all']
    
tag_to_ix = generate_tag_to_ix(keys)
ix_to_tag = {value: key for key, value in tag_to_ix.items()}

tag_to_ix

Wall time: 1.71 s


{'Add_TradeName': 0,
 'Add_Brand': 1,
 'Add_KeyWordTrue': 2,
 'Add_KeyWordFalse': 3,
 'Add_GrapeVarieties': 4,
 'Add_GeoIndication': 5,
 'Add_WineType': 6,
 'Add_BottleSize': 7,
 'Add_Sweetness': 8,
 'Add_WineColor': 9,
 'Add_ClosureType': 10,
 'Add_Certificate': 11,
 'Add_Vintage': 12,
 'Add_Price': 13,
 'Punctuation': 14,
 'Other': 15}

In [4]:
sents = DataGenerator.generate_sents2(
    open(DATASET_PATH, encoding='utf-8').read().split('\n')
)
# sents = sents[:300]
dataset = CustomDataset(
    sents, tag_to_ix, word_to_ix, case_sensitive=CASE_SENSITIVE_VOCAB, prepare_dataset=False, convert_nums2words=USE_NUM2WORDS
)

In [5]:
def features_with_keys(sentence):
    return [{f'A{i}': feature for i, feature in enumerate(features)} for features in sentence]

def compute_features(x_sent, y_sent):
    our_features = sent2features(list(zip(x_sent, y_sent)), freq_dict)
    x_tensor = torch.tensor(dataset.sentence_to_indices(x_sent), dtype=torch.int64)
    final_features = lstm_model.get_lstm_features(x_tensor.to(DEVICE).unsqueeze(0))
    final_features = features_with_keys(final_features.squeeze(0).detach().cpu().numpy())

    for i in range(len(x_sent)):
        final_features[i].update(our_features[i])
    
    return final_features

In [6]:
%%time
X_tensors = (
    torch.tensor(dataset.sentence_to_indices(x_sent), dtype=torch.int64) for x_sent, _ in dataset.raw_data()
)
labels = [y_sent for _, y_sent in dataset.raw_data()]
len(labels)

Wall time: 0 ns


300

In [7]:
%%time
train_sents, val_sents = train_test_split(dataset.raw_data(), test_size=TEST_SIZE)
len(train_sents), len(val_sents)

Wall time: 0 ns


(240, 60)

In [15]:
%%time
with torch.no_grad():
    X_train =(compute_features(x_sent, y_sent) for x_sent, y_sent in train_sents)
    y_train = [y_sent for _, y_sent in train_sents]

    X_val = [compute_features(x_sent, y_sent) for x_sent, y_sent in val_sents]
    y_val = [y_sent for _, y_sent in val_sents]

Wall time: 10.1 s


In [16]:
%%time
with torch.no_grad():
    crf_model = sklearn_crfsuite.CRF(
        algorithm=ALGORITHM,
        c1=C1,
        c2=C2,
        max_iterations=MAX_ITERATIONS,
        all_possible_transitions=ALL_POSSIBLE_TRANSITIONS
    )
    crf_model.fit(X_train, y_train)
eli5.show_weights(crf_model, top=(30, 30))

Wall time: 45.7 s


From \ To,Add_BottleSize,Add_Brand,Add_ClosureType,Add_GeoIndication,Add_GrapeVarieties,Add_KeyWordFalse,Add_KeyWordTrue,Add_Price,Add_Sweetness,Add_TradeName,Add_Vintage,Add_WineColor,Add_WineType,Punctuation
Add_BottleSize,0.862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Add_Brand,-0.099,1.72,0.0,0.0,0.076,0.0,0.0,0.0,0.0,-0.588,-0.078,0.0,0.0,0.0
Add_ClosureType,0.0,0.0,0.648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Add_GeoIndication,0.0,0.0,0.0,0.495,0.0,0.092,-0.331,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Add_GrapeVarieties,0.0,0.0,0.0,0.001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Add_KeyWordFalse,0.183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Add_KeyWordTrue,-0.221,-0.451,0.0,0.0,0.0,0.0,1.043,0.0,0.0,0.0,0.0,0.0,0.0,0.266
Add_Price,-0.063,0.0,0.0,0.0,0.0,0.0,0.0,0.95,0.0,0.0,-0.402,0.0,0.0,0.0
Add_Sweetness,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Add_TradeName,0.0,0.701,0.0,0.0,0.0,0.0,-0.117,0.0,0.0,0.81,-0.008,0.0,0.0,0.336


In [17]:
%%time
with torch.no_grad():
    y_pred = crf_model.predict(X_val)
    marginals = crf_model.predict_marginals(X_val)

Wall time: 90 ms


In [20]:
unk_foreach_tag = count_unk_foreach_tag(X_tensors, labels, keys, dataset.word_to_ix[dataset.unk])

In [21]:
confs = compute_model_confidence(marginals)
prob_table = DataGenerator.generate_probability_table(marginals, val_sents)

In [22]:
%%time
test_eval = [list(zip(sentence, tags, y_pred[index])) for index, (sentence, tags) in enumerate(val_sents)]

Wall time: 0 ns


In [23]:
%%time
run_params = {
    'dataset_path': DATASET_PATH,
    'lstm_model_path': LSTM_MODEL_PATH,
    'vocab_path': VOCAB_PATH,
    'case_sensitive_vocab': CASE_SENSITIVE_VOCAB,
    'dictionary_path': DICTIONARY_PATH,
    'datainfo_path': DATAINFO_PATH,
    'device': DEVICE,
    'model_name': MODEL_NAME,
    'algorithm': ALGORITHM,
    'c1': C1,
    'c2': C2 ,
    'max_iterations': MAX_ITERATIONS,
    'all_possible_transitions': ALL_POSSIBLE_TRANSITIONS,
    'test_size': TEST_SIZE,
    'runname': RUN_NAME,
    'start_time': START_TIME,
    'output_dir': OUTPUT_DIR,
    'models_confidence': np.mean(confs),
    'unk_foreach_tag': json.dumps(unk_foreach_tag),
    'prob_table': prob_table,
    'use_num2words': USE_NUM2WORDS
}

Wall time: 0 ns


In [ ]:
%%time
log_mlflow_on_train(
    run_params=run_params,
    model=crf_model,
    y_true=y_val,
    y_pred=y_pred,
    test_eval=test_eval
)